In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import os
import time , sys

In [2]:
if  getattr(sys, 'frozen', False): 
    chromedriver_path = os.path.join(sys._MEIPASS, "chromedriver.exe")
    dr = webdriver.Chrome(chromedriver_path)
else:
    dr = webdriver.Chrome()

In [3]:
dr.implicitly_wait(5)

In [4]:
options = webdriver.ChromeOptions()
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 6.1; WOW64; Trident/7.0; rv:11.0) like Gecko")

In [5]:
login_site = 'http://clc.chosun.ac.kr/ilos/main/member/login_form.acl'
m='0'
now_site = dr.current_url
dr.get(login_site)
if now_site == login_site:
    print('로그인 후 온라인 강의 학습버튼 클릭 직전으로 이동')
else:
    print('온라인 강의 학습하기 버튼 클릭 직전으로 이동')

온라인 강의 학습하기 버튼 클릭 직전으로 이동


In [6]:
#강의 리스트
def start():
    input('강의 목록으로 이동 했다면 엔터를 눌러 실행')
    try:
        lec_list = dr.find_elements_by_class_name('lecture-box')
        lec_num = len(lec_list)
        for i in (0,lec_num-1):
            print(lec_list[(i)].text)
        lec_sys()
    except:
        try:
            lec_list = dr.find_elements_by_id('online_mode_wrap')
            lec_on_sys()
        except:
            print('온라인 강의 학습하기 버튼 클릭 직전으로 가야함\n사이버강의는 전체 차수가 보이는 기본화면으로')

dr.find_element_by_xpath('//*[@id="K014"]').click()

lec_list = dr.find_elements_by_class_name('lecture-box')
lec_num = len(lec_list)
lec_list[1]

dr.switch_to.default_content()

In [7]:
#일반 강의 시스템
def lec_sys():
    lec_list = dr.find_elements_by_class_name('lecture-box')
    lec_num = len(lec_list)
    try:
        for i in range(0,lec_num):
            lec_mov((i))
            print('강의 %d 개 끝' %(i+1))
        lec_quit()
    except:
        print('에러')

In [8]:
#온라인 강의 시스템
def lec_on_sys():
    print(dr.find_element_by_xpath('//*[@id="online_mode_lect_nm"]/span/span[1]').text)
    dr.find_element_by_xpath('//*[@id="K014"]').click()
    lec_on_mov()

In [9]:
def lec_on_mov():
    per = dr.find_element_by_xpath('//*[@id="per_text"]')
    print(dr.find_element_by_xpath('//*[@id="lecture-1"]/div').text)
    if per == '100%':
        print('이미 수강함')
        try:
            dr.find_element_by_xpath('//*[@id="online_content_nav_next"]').click()
        except:
            print('마지막 강의입니다.')
            lec_quit()
    else:
        dr.find_element_by_xpath('//*[@id="lecture-1"]/div/ul/li[2]/img').click()
        lec_on_cert()

print(dr.find_element_by_xpath('//*[@id="lecture-1"]/div').text)

In [10]:
def lec_on_cert():
    try:
        dr.find_element_by_xpath('//*[@id="ui-id-2"]')
        print('인증 자동은 없으므로 개인이 해야함')
        AA = input('인증을 완료하여서 재생화면으로 넘어갔다면 1+엔터, 처음부터 다시는 2+엔터')
        if AA == '1':
            print('강의 재생 시작, 이후 작업은 자동으로 수행할 것임')
            lec_on_play()
        elif AA == '2':
            pass
        else:
            lec_on_cert()
    except:
        dr.find_element_by_xpath('//*[@id="lecture-1"]/div/ul/li[2]/img').click()
        lec_on_play()

In [11]:
def lec_on_play():
    try:
        time.sleep(3)
        dr.switch_to.default_content()
        dr.switch_to.frame(dr.find_element_by_xpath('//*[@id="contentViewer"]'))
        try:
            dr.find_element_by_xpath('//*[@id="test_player"]/button').click()
        except:
            dr.find_element_by_xpath('//*[@id="front-screen"]/div/div[2]/div[1]/div').click()
        lec_time()
    except:
        print('재생환경 에러')

In [12]:
def lec_on_time():
    dr.switch_to.default_content()
    dr.switch_to.frame(dr.find_element_by_xpath('//*[@id="contentViewer"]'))
    try:
        et = dr.find_element_by_xpath('//*[@id="play-controller"]/div/div/div[2]/div[4]/span[3]').text
        nt = dr.find_element_by_xpath('//*[@id="play-controller"]/div/div/div[2]/div[4]/span[1]').text
    except:
        et = dr.find_element_by_xpath('//*[@id="test_player"]/div[4]/div[2]/span[2]').text
        nt = dr.find_element_by_xpath('//*[@id="test_player"]/div[4]/div[4]/span[2]').text
    if et == nt:
        dr.switch_to.default_content()
        dr.find_element_by_xpath('//*[@id="next_"]').click()
        try:
            dr.switch_to_alert().accept()
            dr.find_element_by_xpath('//*[@id="close_"]').click()
            try:
                dr.switch_to_alert().accept()
            except:
                pass
        except:
            lec_play()
    else:
        time.sleep(10)
        lec_time()

In [13]:
#강의 진도 확인
def lec_mov(a) :
    lec_list = dr.find_elements_by_class_name('lecture-box')
    if lec_list[(a)].find_element_by_id('per_text').text =='100%':
        print('이미 수강한 강의')
        pass
    else:
        try:
            bool(lec_list[(a)].find_element_by_class_name('view'))
            lec_start((a))
        except:
            print('수강 기간이 아님')

In [20]:
#강의 종료 인풋
def lec_quit():
    AB=input('강의 종료됨 정상 종료라면 1+엔터로 종료를, 아니면 2+엔터로 재시작')
    if AB =='1':
        m='1'
        quit()
        exit()
    elif AB =='2':
        pass
    else:
        lec_quit()

In [15]:
#일반강의 학습시작
def lec_start(a):
    lec_list = dr.find_elements_by_class_name('lecture-box')
    lec_num = len(lec_list)
    print(lec_list[(a)])
    dr.execute_script("window.scrollTo(0,document.body.scrollHeight)")
    try:
        lec_list[(a)].find_element_by_class_name('view').click()
        try:
            dr.switch_to_alert().accept()
        except:
            pass
        lec_play()
    except:
        print('시작 에러')

In [16]:
#일반강의 플레이
def lec_play():
    try:
        time.sleep(4)
        dr.switch_to.default_content()
        dr.switch_to.frame(dr.find_element_by_xpath('//*[@id="contentViewer"]'))
        try:
            dr.find_element_by_xpath('//*[@id="test_player"]/button').click()
        except:
            dr.find_element_by_xpath('//*[@id="front-screen"]/div/div[2]/div[1]/div').click()
        lec_time()
    except:
        print('재생환경 에러')

In [17]:
#일반강의 시간체크/종료
def lec_time():
    dr.switch_to.default_content()
    dr.switch_to.frame(dr.find_element_by_xpath('//*[@id="contentViewer"]'))
    try:
        et = dr.find_element_by_xpath('//*[@id="play-controller"]/div/div/div[2]/div[4]/span[3]').text
        nt = dr.find_element_by_xpath('//*[@id="play-controller"]/div/div/div[2]/div[4]/span[1]').text
    except:
        et = dr.find_element_by_xpath('//*[@id="test_player"]/div[4]/div[2]/span[2]').text
        nt = dr.find_element_by_xpath('//*[@id="test_player"]/div[4]/div[4]/span[2]').text
    if et == nt:
        dr.switch_to.default_content()
        dr.find_element_by_xpath('//*[@id="next_"]').click()
        try:
            dr.switch_to_alert().accept()
            dr.find_element_by_xpath('//*[@id="close_"]').click()
            try:
                dr.switch_to_alert().accept()
            except:
                pass
        except:
            lec_play()
    else:
        time.sleep(10)
        lec_time()

In [18]:
while not m =='1':
    start()

KeyboardInterrupt: Interrupted by user